In [3]:
import requests
import requests_cache
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
import time
import json
import re
import numpy as np
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from datetime import datetime, timedelta # Dùng để xử lý dữ liệu thời gian

import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không

In [4]:
sleep_time = 1 # Đơn vị: giây

In [5]:
ignoreRes = lambda s: '"incomplete_results":true' not in s.text
requests_cache.install_cache(expire_after=-1, filter_fn=ignoreRes)

In [8]:
def get_user_id(init_id, limit, client_id):
    
    list_user_id = [init_id]# User id list
    set_user_id = set(list_user_id) # User id set
    index = 0 # Used for tracing more users
    
    # Cấu hình retry của request nếu gặp lỗi
    retries = Retry(total=6, backoff_factor=sleep_time, status_forcelist=[ 500, 502, 503, 504 ])
    s = requests.Session()
    s.mount('https://', HTTPAdapter(max_retries=retries))
    
    while len(list_user_id) < limit:
        # Không còn user id để trace
        if index == len(list_user_id):
            break
        
        # Lấy dữ liệu với url tương ứng
        url = f'https://api-v2.soundcloud.com/users/{list_user_id[index]}/followings?client_id={client_id}'
        print('Get:', url)
        
        try:
            re= s.get(url)
        except:
            if re.status_code != 200 and re.status_code != 429:
                print("Error! Status code:",re.status_code)
                continue
        
        # Vòng lặp kiểm tra request đã sẵn sàng để crawl chưa
        canGet = False
        while not canGet:
            # Nếu request không được thì sleep rồi request lại
            if not re.ok:
                time.sleep(sleep_time)
                re = requests.get(url)
                print('Not ok')
                continue
            
            # Kiểm tra request có bị timeout
            # Nếu có sleep rồi request lại
            if re.status_code == 504:
                time.sleep(sleep_time)
                re = requests.get(url)
                print('Time out')
                continue
                
            if re.status_code == 429:
                print('Exceeds an API rate limit')
                return list_user_id
                
            # Đổi Json string thành python dictionary
            json_pydata = json.loads(re.text)
            
            # Request sẵn sàng để crawl
            canGet = True
            
        print('Pass:', url)
            
        # Thêm user id
        for user in json_pydata['collection']:
            if user['id'] not in set_user_id:
                set_user_id.add(user['id'])
                list_user_id.append(user['id'])
        
        index = index + 1
        
    return list_user_id

In [12]:
list_user_id = get_user_id(630253005, 10000, '3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn')

Get: https://api-v2.soundcloud.com/users/630253005/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/630253005/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/217441590/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/217441590/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/324531068/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/324531068/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/525378972/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/525378972/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/9855085/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/9855085/fol

Pass: https://api-v2.soundcloud.com/users/148539361/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/72572274/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/72572274/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/83529969/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/83529969/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/75063811/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/75063811/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/121708922/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/121708922/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/26057743/follow

Pass: https://api-v2.soundcloud.com/users/69495898/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/338583/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/338583/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/51239523/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/51239523/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/142105854/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/142105854/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/376046318/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/376046318/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/252088355/followin

Pass: https://api-v2.soundcloud.com/users/947113846/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/71180310/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/71180310/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/1003295896/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/1003295896/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/574405968/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/574405968/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/378147863/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/378147863/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/820384885

Pass: https://api-v2.soundcloud.com/users/574078455/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/497618937/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/497618937/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/315596869/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/315596869/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/159142211/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/159142211/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/909658033/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/909658033/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/14168495/

...

Pass: https://api-v2.soundcloud.com/users/70533565/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/184512166/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/184512166/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/128178923/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/128178923/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/315192020/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/315192020/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/205854877/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/205854877/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/245290661/

Pass: https://api-v2.soundcloud.com/users/360782528/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/133550136/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/133550136/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/607039/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/607039/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/1036179259/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/1036179259/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/69745433/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/69745433/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/8983540/followi

Pass: https://api-v2.soundcloud.com/users/180994642/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/266350426/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/266350426/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/623217717/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/623217717/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/72509893/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/72509893/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/798645148/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/798645148/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/32286436/fo

Pass: https://api-v2.soundcloud.com/users/27124455/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/23433070/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/23433070/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/96005780/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/96005780/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/211793852/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/211793852/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/530813127/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/530813127/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/165165172/foll

Pass: https://api-v2.soundcloud.com/users/1207292/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/122956334/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/122956334/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/19506990/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/19506990/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/223510065/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/223510065/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/141640712/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Pass: https://api-v2.soundcloud.com/users/141640712/followings?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Get: https://api-v2.soundcloud.com/users/247955/follow

In [13]:
np.savetxt('user_id.txt', list_user_id, fmt='%d')